# Installating Julia/IJulia on winpython2015-03-27 and after via a Notebook
###1 - Downloading and Installing the right Julia binary in the right place

In [1]:
import os
import sys
import io

In [2]:
# downloading julia (32Mo, may take 1 minute or 2)
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2
if 'amd64' in sys.version.lower():
    julia_binary="julia-0.3.7-win64.exe"
    julia_url="http://s3.amazonaws.com/julialang/bin/winnt/x64/0.3/julia-0.3.7-win64.exe"
    hashes=("90dedbbee5deb569c613543cc709af85", "813f049445421b9531cb6a3a4a2871fdbe1ca5d0" )
else:
    julia_binary="julia-0.3.7-win32.exe"
    julia_url="http://s3.amazonaws.com/julialang/bin/winnt/x86/0.3/julia-0.3.7-win32.exe"
    hashes=("bd2237758fb2a034174249010cf7ba33", "4319c95c38622792b112478d240d2586e08430f0")
    
julia_installer=os.environ["WINPYDIR"]+"\\..\\tools\\"+julia_binary
os.environ["julia_installer"]=julia_installer
g = urllib2.urlopen(julia_url) 
with io.open(julia_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [3]:
#checking it's there
!dir %julia_installer%

 Le volume dans le lecteur D s'appelle DATA
 Le num‚ro de s‚rie du volume est 10BD-2ADB

 R‚pertoire de D:\WinPython\basedir34\build\winpython-3.4.3.amd64\tools

27/03/2015  18:51        36ÿ789ÿ424 julia-0.3.7-win64.exe
               1 fichier(s)       36ÿ789ÿ424 octets
               0 R‚p(s)  120ÿ517ÿ468ÿ160 octets libres


In [4]:
# checking it's the official julia0.3.2
import hashlib
def give_hash(of_file, with_this):
    with io.open(julia_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*12+"MD5"+" "*(32-12-3)+" "+" "*15+"SHA-1"+" "*(40-15-5)+"\n"+"-"*32+" "+"-"*40)

print ("%s %s %s" % (give_hash(julia_installer, hashlib.md5) , give_hash(julia_installer, hashlib.sha1),julia_installer))
assert give_hash(julia_installer, hashlib.md5) == hashes[0]
assert give_hash(julia_installer, hashlib.sha1) == hashes[1]

            MD5                                 SHA-1                    
-------------------------------- ----------------------------------------
90dedbbee5deb569c613543cc709af85 813f049445421b9531cb6a3a4a2871fdbe1ca5d0 D:\WinPython\basedir34\build\winpython-3.4.3.amd64\python-3.4.3.amd64\..\tools\julia-0.3.7-win64.exe


In [5]:
os.environ["JULIA_HOME"] = os.environ["WINPYDIR"]+"\\..\\tools\\Julia\\bin\\"
os.environ["JULIA_EXE"]="julia.exe"
os.environ["JULIA"]=os.environ["JULIA_HOME"]+os.environ["JULIA_EXE"]
# for installation we need this
os.environ["JULIAROOT"]=os.path.join(os.path.split(os.environ["WINPYDIR"])[0]  , 'tools','julia' )  

In [6]:
# let's install it (add a  /S before /D if you want silence mode installation)
#nullsoft installers don't accept . or .. conventions

# If you are "USB life style", or multi-winpython
#   ==> UN-CLICK the OPTION 'CREATE a StartMenuFolder and Shortcut' <== (when it will show up)
!start cmd /C %julia_installer% /D=%JULIAROOT%


<img src="https://raw.githubusercontent.com/stonebig/winpython_afterdoc/master/examples/images/julia_setup_unclick_all.GIF">

###2 - Initialize Julia , IJulia, and make them link to winpython

In [12]:
# Writing a julia initial run script, for convenience
bat_text = r"""
@echo off
set WINPYDIR=%~dp0..\blablaPYTHON
set WINPYVER=blablaWINPYVER
set HOME=%WINPYDIR%\..\settings
set PATH=%WINPYDIR%\Lib\site-packages\PyQt4;%WINPYDIR%\;%WINPYDIR%\DLLs;%WINPYDIR%\Scripts;%WINPYDIR%\..\tools;%WINPYDIR%\..\tools\mingw32\bin;%PATH%;%WINPYDIR%\..\tools\TortoiseHg

set JULIA_HOME=%WINPYDIR%\..\tools\Julia\bin\
if  exist "%JULIA_HOME%" goto julia_next
echo --------------------
echo First install Julia in \tools\Julia of winpython
echo suggestion : don't create Julia shortcuts, nor menu, nor desktop icons 
echo (they would create a .julia in your home directory rather than here)
echo When it will be done, launch again this .bat

if not exist "%JULIA_HOME%" goto julia_end

:julia_next
set SYS_PATH=%PATH%
set PATH=%JULIA_HOME%;%SYS_PATH%

set JULIA_EXE=julia.exe
set JULIA=%JULIA_HOME%%JULIA_EXE%
set private_libdir=bin
if not exist "%JULIA_HOME%..\lib\julia\sys.ji" ( ^
echo "Preparing Julia for first launch. This may take a while"  
echo "You may see two git related errors. This is completely normal"  
cd "%JULIA_HOME%..\share\julia\base"  
"%JULIA%" --build "%JULIA_HOME%..\lib\julia\sys0" sysimg.jl  
"%JULIA%" --build "%JULIA_HOME%..\lib\julia\sys" -J sys0.ji sysimg.jl  
popd && pushd "%cd%" )

echo "julia!"
echo --------------------
echo to install Ijulia for Winpython (the first time) :
echo type 'julia'
echo type in Julia prompt 'Pkg.add("IJulia")'
echo type in Julia prompt 'Pkg.add("PyCall")'
echo type in Julia prompt 'Pkg.add("PyPlot")'
echo type in Julia prompt 'Pkg.add("Interact")'
echo type in Julia prompt 'Pkg.add("Compose")'
echo type in Julia prompt 'Pkg.add("SymPy")'
echo type 'Ctrl + 'D' to quit Julia 
echo nota : type 'help()' to get help in Julia
echo --------------------
rem 2014-08-23 refinement
echo or let me do it now
pause
echo Pkg.add("IJulia");>this_is_temporary.jl
echo Pkg.add("PyCall");>>this_is_temporary.jl
echo Pkg.add("PyPlot");>>this_is_temporary.jl
echo Pkg.add("Interact");>>this_is_temporary.jl
echo Pkg.add("Compose");>>this_is_temporary.jl
echo Pkg.add("SymPy");>>this_is_temporary.jl
@echo on
julia this_is_temporary.jl  
@echo off

echo Julia installed
echo use the "ijulia_launcher.bat" script to launch Ijulia directly
pause
:julia_end
rem cmd.exe /k
"""
bat_text = bat_text.replace("blablaPYTHON",os.path.split(os.environ["WINPYDIR"])[1])
bat_text = bat_text.replace("blablaWINPYVER",os.environ["WINPYVER"])
julia_initializer_bat=os.environ["WINPYDIR"]+"\\..\\scripts\\initialize_julia_once.bat"
if sys.version_info[0] == 3:
    with io.open(julia_initializer_bat, 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in bat_text.splitlines():
            f.write('%s\n' %  line  )
else:
    with io.open(julia_initializer_bat, 'wb'  ) as f:
        for line in bat_text.splitlines():
            f.write('%s\r\n' %  line.encode(sys.getdefaultencoding()) )

In [8]:
# let's initialize Julia and install "IJulia", "PyCall", and "PyPlot" Julia modules with this .bat just created
# may take about 10 minutes (Julia pre-compiles itself and download a lot of things)
!start cmd /C %WINPYDIR%\\..\\scripts\\initialize_julia_once.bat

In [9]:
# let's launch Ijulia for Ipython now
# Writing a julia initial run script, for convenience
bat_text = r"""
@echo off

set WINPYDIR=%~dp0..\blablaPYTHON
set WINPYVER=blablaWINPYVER
set HOME=%WINPYDIR%\..\settings
set PATH=%WINPYDIR%\Lib\site-packages\PyQt4;%WINPYDIR%\;%WINPYDIR%\DLLs;%WINPYDIR%\Scripts;%WINPYDIR%\..\tools;%WINPYDIR%\..\tools\mingw32\bin;%PATH%;%WINPYDIR%\..\tools\TortoiseHg

set JULIA_HOME=%WINPYDIR%\..\tools\Julia\bin\

set SYS_PATH=%PATH%
set PATH=%JULIA_HOME%;%SYS_PATH%

set JULIA_EXE=julia.exe
set JULIA=%JULIA_HOME%%JULIA_EXE%

Ipython notebook --profile julia
echo to use julia_magic from Ipython, type "Ipython notebook" instead.
:julia_end
cmd.exe /k
"""
bat_text = bat_text.replace("blablaPYTHON",os.path.split(os.environ["WINPYDIR"])[1])
bat_text = bat_text.replace("blablaWINPYVER",os.environ["WINPYVER"])

ijulia_launcher_bat=os.environ["WINPYDIR"]+"\\..\\scripts\\ijulia_launcher.bat"
if sys.version_info[0] == 3:
    with io.open(ijulia_launcher_bat, 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in bat_text.splitlines():
            f.write('%s\n' %  line  )
else:
    with io.open(ijulia_launcher_bat, 'wb'  ) as f:
        for line in bat_text.splitlines():
            f.write('%s\r\n' %  line.encode(sys.getdefaultencoding()) )


###3 - Launching IJulia 

either from here

either with your new winpython**\scripts\ijulia_launcher.bat

In [ ]:
!start cmd /C %WINPYDIR%\\..\\scripts\\ijulia_launcher.bat

And now, you should see a IJulia notebook starting next to this IPython Notebook

Check a new Ijulia notebook works with a few julia commands
````
println(2+2);
versioninfo(true)
````

<img src="https://raw.githubusercontent.com/stonebig/winpython_afterdoc/master/examples/images/IJulia_first_cry.gif">

###4 - Julia Magic ?
What preceeds should work with all previous version of winpython

This part may only work for recent winpythons 

In [13]:
# first, we must patch the path
import os
os.environ["JULIA_HOME"] = os.environ["WINPYDIR"]+"\\..\\tools\\Julia\\bin\\" # \\bin"
os.environ["JULIA_EXE"]="julia.exe"
if "\\julia\\" not in  os.environ["PATH"].lower():
    os.environ["JULIA"]=os.environ["JULIA_HOME"]+""+os.environ["JULIA_EXE"]
    os.environ["PATH"]  =os.environ["JULIA_HOME"]+";"+os.environ["PATH"]
#now we can
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [14]:
# don't worry if you see "Julia error ? Failed to initialize PyCall package" and continue as usual 
%julia @pyimport matplotlib.pyplot as plt
%julia @pyimport numpy as np

In [15]:
%%julia

# Note how we mix numpy and julia:
t = linspace(0, 2*pi, 1000); # use the julia linspace
s = sin(3 * t + 4 * np.cos(2 * t)); # use the numpy cosine and julia sine

fig = plt.gcf()  # **** WATCH THIS VARIABLE ****
plt.plot(t, s, color="red", linewidth=2.0, linestyle="--")

#5 - To Un-install  / Re-install Julia and Ijulia (or other trouble-shooting)

If the Ijulia doesn't work because of a "dead kernel" the comes every minute
(or if you just want to un-install Julia 0.3)


- stop all Ipython instances that are running 

- launch  winpython**\tools\Julia\uninstall.exe

- delete the directory winpython**\settings\.julia

- remove winpython**\settings\.ipython\default_julia

(maybe check you don't have an annoying c:\users\your_windows_profile\.ipython\default_julia)

- re-install


In [16]:
import julia
j=julia.Julia()
j.eval("1 +31")
j.eval("sqrt(1 +31)")

5.656854249492381